In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample
from sklearn.metrics import roc_auc_score

## Regresión Logística simple

### Accuracy

In [17]:
df = pd.read_csv("Default.csv")

In [18]:
X = df[["balance", "income", "student"]].copy()
X["student"] = X["student"].map({"No": 0, "Yes": 1}).astype(float)
X["balance"] = X["balance"].astype(float)
X["income"] = X["income"].astype(float)

y = df["default"].map({"No": 0, "Yes": 1})

Lr = LogisticRegression(max_iter=1000)
Lr.fit(X, y)

y_pred = Lr.predict(X)

accuracy = (y_pred == y).mean()

print("Accuracy:", accuracy)

Accuracy: 0.9671


### AUC

In [19]:
y_prob = Lr.predict_proba(X)[:, 1]  

auc_score = roc_auc_score(y, y_prob)

print("AUC:", auc_score)

AUC: 0.9070324073944639


### Bagging

### Accuracy

In [16]:
df = pd.read_csv("Default.csv")
y = df["default"].map({"No": 0, "Yes": 1})
df["student"] = df["student"].map({"No": 0, "Yes": 1})
X = df.drop(columns=["default"])

all_features = ["balance", "income","student"] 
B = 5000

# Matriz para guardar predicciones de clase de cada modelo
y_preds = np.zeros((B, len(df)), dtype=int)

models = []
selected_features_list = []

for b in range(B):
    # Seleccionar 2 columnas al azar 
    selected_features = np.random.choice(all_features, size=2, replace=False)
    selected_features_list.append(selected_features)
    X_b = X[selected_features]
    
    # Bootstrap de filas
    X_b_resampled, y_b = resample(X_b, y, replace=True, n_samples=5000)
    
    # Entrenar árbol de decisión
    tree = DecisionTreeClassifier()
    tree.fit(X_b_resampled, y_b)
    models.append(tree)

# Predecir usando cada modelo sobre sus columnas correspondientes
for i in range(B):
    y_preds[i, :] = models[i].predict(X[selected_features_list[i]])

y_pred_final = mode(y_preds, axis=0).mode[0]

# Calcular accuracy
accuracy = (y_pred_final == y).mean()
print("Accuracy:", accuracy)

C:\Users\Michelle Trigo\AppData\Local\Temp\ipykernel_11248\3065813103.py:33: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  y_pred_final = mode(y_preds, axis=0).mode[0]


Accuracy: 0.9846


## Comparación 
Con la regresión logística simple, el modelo logró un accuracy de 0.9671. Este modelo es bastante bueno, pero es lineal, es decir, busca una relación directa entre las variables (balance, ingreso y ser estudiante) y la probabilidad de default. Esto puede limitar su capacidad de capturar relaciones más complejas entre los datos.

Por otro lado, al usar bagging con árboles de decisión, el accuracy subió a 0.9846. Bagging combina muchos árboles de decisión entrenados con diferentes muestras y características, y luego hace una votación por mayoría. Esto permite al modelo capturar patrones más complejos y reducir errores que un solo árbol o un modelo lineal podría cometer.

# Conclusión
Ambos modelos funcionan muy bien, pero bagging supera a la regresión logística en este caso porque es más flexible y robusto. La mejora en accuracy muestra que, para este conjunto de datos, combinar muchos árboles ayuda a predecir mejor el default que usar un solo modelo lineal.